In [4]:
import matplotlib.pyplot as plt
import numpy as np

from tracking.filters.jipdaf import PDA, Track, build_clusters, track_betas
from tracking.util.interpolator import LinearInterpolator, NodeCollection, SplineInterpolator
from tracking.util.metrics import RadarGenerator, Scan, TrackLog, PoissonClutter
from tracking.util.path import Path2D, PathFactory
from tracking.visualizer import TrackVisualizer
from tracking.filters.track_manager import TrackManager, TrackingVisualizer



In [20]:
TMAX = 500

nodes = [
    # NodeCollection(np.array([[-400, -100], [400,  100]]), [0, TMAX]),
    NodeCollection(np.array([[-400, 100],  [400, -100]]), [0, TMAX]),
    NodeCollection(np.array([[-600, 0], [600, 0]]), [0, TMAX]),
    NodeCollection(np.array([[3, 600], [-3, -600]]), [0, TMAX]),
    NodeCollection(np.array([[-3, 600], [3, -600]]), [0, TMAX]),
]
node = NodeCollection(np.array(
    [[-400, -100], [-200, 25], [0, 0], [200, -25], [400, 100]]), np.linspace(0, TMAX, 5))
factory = PathFactory()
paths = factory.create(nodes, LinearInterpolator)
paths += factory.create([node], SplineInterpolator)

nprobes = 200
probexy = np.zeros(nprobes)
probetimes = np.linspace(0.0, TMAX-0.1, nprobes)
radar = RadarGenerator(paths, 0.3, 1)
clutter_model = PoissonClutter([-500, 400, -250, 250], 0.00001)
scans = radar.make_scans_series(
    np.array([probetimes, probexy, probexy]).T, clutter_model)

manager = TrackManager()

nsteps = 60000
time = 0
dt = 0.05

working_scans = scans.copy()

for _ in range(nsteps):
    time += dt

    manager.predict_tracks(time, dt)
    
    if time+dt >= working_scans[0].time:
        time = working_scans[0].time

        manager.update_tracks(working_scans[0], time)
        manager.one_point_init(vmax=5, p0=0.8)
        manager.delete_false_tracks(.05)
        
        working_scans.pop(0)
        if len(working_scans) == 0:
            print("Processed all scans")
            break

updating tracks
track deletion
updating tracks
number of tracks in cluster: 1
probability of existence: 0.8501840320703344
clutter density: 0.0
number of tracks in cluster: 1
probability of existence: 0.8501840320703344
clutter density: 0.0
number of tracks in cluster: 1
probability of existence: 0.8501840320703344
clutter density: 0.0
track deletion
updating tracks
number of tracks in cluster: 2
probability of existence: 0.10064539668376941
clutter density: 2.5797676498348447e-06
probability of existence: 0.09996974859414232
clutter density: 2.5797676498348447e-06
number of tracks in cluster: 1
probability of existence: 0.0999697478671914
clutter density: 0.0
number of tracks in cluster: 1
probability of existence: 0.9998828872832553
clutter density: 8.31549255331164e-06
number of tracks in cluster: 1
probability of existence: 0.999849620871594
clutter density: 8.31549255331164e-06
number of tracks in cluster: 2
probability of existence: 0.9966423987526485
clutter density: 0.000179069

In [22]:
%matplotlib qt 
renderer = TrackingVisualizer(manager.logger)
plt.ion()
plt.figure()
plot = plt.subplot(1, 2, 1)
    
renderer.set_number_rendered_epochs(40)
renderer.filter_tracks_by_length(4)
while renderer.advance_epoch():
    plot.cla()
    renderer.render(plot)
    plt.show(block=False)
    plt.pause(0.1)
#for i in range(145):
#    renderer.advance_epoch()
renderer.render(plot)
